In [2]:
import pandas as pd
import datetime
from os import listdir
import os, os.path
from os.path import isfile, join
import numpy as np
import csv
import re
import pandas as pd
import pm4py
from pm4py.objects.conversion.log import converter as log_converter
from pm4py.objects.log.importer.xes import importer as xes_importer
from pm4py.objects.log.exporter.xes import exporter as xes_exporter
from pm4py.objects.log.util import dataframe_utils
import matplotlib.pyplot as plt
import seaborn as sns

In [1]:
df_categorias_autores=pd.read_csv("logCT2.csv",index_col=0)
df_categorias_autores['Category'].unique()

NameError: name 'pd' is not defined

In [3]:
winners=['A1 2015',"A2 2015","A13 2017","A18 2017","A22 2017","A4 2019","A12 2019","A1 2020","A2 2020"]
df_categorias_autores["repYear"]=[rep+" "+str(year) for rep,year in zip(df_categorias_autores['Report'],df_categorias_autores['Year'])]

In [4]:
df_categorias_autores.head()

,case:concept:name,concept:name,Specific operation,time:timestamp,Report,Question,Year,Category,Type,repYear
0,A1 P2 BPI 2020,Calculate cycle time,Calculate cycle time of the whole process for ...,2021-10-19 12:47:13.651481,A1,P2,2020,Academic,Analyze Cycle Time,A1 2020
1,A1 P2 BPI 2020,Compare cycle time,,2021-10-19 12:47:13.659459,A1,P2,2020,Academic,Analyze Cycle Time,A1 2020
2,A2 P2 BPI 2020,Calculate cycle time,Get cycle time from the previous question,2021-10-19 12:47:13.666495,A2,P2,2020,Student,Analyze Cycle Time,A2 2020
3,A2 P2 BPI 2020,Compare cycle time,,2021-10-19 12:47:13.666495,A2,P2,2020,Student,Analyze Cycle Time,A2 2020
4,A7 P2 BPI 2020,Calculate throughput,,2021-10-19 12:47:13.682150,A7,P2,2020,Academic,Analyze Cycle Time,A7 2020


In [5]:
df_analisis = df_categorias_autores.groupby(by=['case:concept:name'])
columns=['case:concept:name','Operations','Winner', 'Year', 'Question']
df_agrupado = pd.DataFrame(columns=columns)
for name, group in df_analisis:
#     org = group.iloc[0]['Organization']
    year = group.iloc[0]['Year']
    question = group.iloc[0]['Question']
    lista = group['concept:name'].to_list()
    if(group.iloc[0]['repYear'] in winners):
        win = 'W'
    else:
        win = 'NW'
        
    new_row={'case:concept:name':name, 'Operations':lista, 'Winner':win, 'Year':year, 'Question':question} 
    df_agrupado = df_agrupado.append(new_row, ignore_index=True)

In [6]:
df_agrupado

,case:concept:name,Operations,Winner,Year,Question
0,A1 P1 BPI 2017,"[Calculate cycle time, Calculate cycle time, F...",NW,2017,P1
1,A1 P1 BPI 2020,"[Filter traces, Calculate cycle time, Calculat...",W,2020,P1
2,A1 P2 BPI 2019,"[Group events by time, Calculate cycle time, R...",NW,2019,P2
3,A1 P2 BPI 2020,"[Calculate cycle time, Compare cycle time]",W,2020,P2
4,A1 P5 BPI 2015,"[Calculate cycle time, Identify organizational...",W,2015,P5
...,...,...,...,...,...
121,A8 P1 BPI 2017,"[Filter traces, Calculate waiting time, Repres...",NW,2017,P1
122,A8 P5 BPI 2015,"[Calculate cycle time, Compare cycle time, Gro...",NW,2015,P5
123,A9 P1 BPI 2017,"[Calculate processing time, Calculate waiting ...",NW,2017,P1
124,A9 P2 BPI 2019,"[Filter traces, Calculate cycle time, Identify...",NW,2019,P2


In [8]:
def overlapIndex(list1,list2):
    set1=set(list1)
    set2=set(list2)
    intersection = set1.intersection(set2)
    len1=len(list(set1))
    len2=len(list(set2))
    overlapIndex=float(len(list(intersection)))/min([len1,len2])
    return overlapIndex

In [9]:
df2 = df_agrupado.groupby(by=['Year', 'Question'])
columns=['analisis','winner','overlap']
df_overlap = pd.DataFrame(columns=columns)

for name, group in df2:
    for i in range(len(group)-1):
        
        for j in range(i+1,len(group)-1):
            grupo = group.iloc[i]['Operations']
            analisis = group.iloc[i]['case:concept:name'] + "-" + group.iloc[j]['case:concept:name']

            win = group.iloc[i]['Winner'] + "-" + group.iloc[j]['Winner']
            overlap = overlapIndex(grupo,group.iloc[j]['Operations'])
            new_row={'analisis':analisis,'winner':win,'overlap': overlap} 
            df_overlap = df_overlap.append(new_row, ignore_index=True)

In [21]:
df_overlap[df_overlap['winner'] == 'W-W']

,analisis,winner,overlap
0,A1 P5 BPI 2015-A2 P5 BPI 2015,W-W,0.250000
102,A13 P1 BPI 2017-A18 P1 BPI 2017,W-W,0.333333
106,A13 P1 BPI 2017-A22 P1 BPI 2017,W-W,0.500000
166,A18 P1 BPI 2017-A22 P1 BPI 2017,W-W,0.333333
250,A12 P2 BPI 2019-A4 P2 BPI 2019,W-W,0.800000
280,A1 P1 BPI 2020-A2 P1 BPI 2020,W-W,1.000000
432,A1 P2 BPI 2020-A2 P2 BPI 2020,W-W,1.000000
689,A1 P5 BPI 2020-A2 P5 BPI 2020,W-W,1.000000
806,A1 P6 BPI 2020-A2 P6 BPI 2020,W-W,0.500000


In [14]:
df_overlap = df_overlap.replace({'W-NW':'NW-W'})

In [15]:
df_overlap.groupby(by=['winner']).mean()

,overlap
winner,
NW-NW,0.494577
NW-W,0.602525
W-W,0.635185


In [16]:
df_overlap.groupby(by=['winner'])['overlap'].var(ddof=0)

winner
NW-NW    0.114217
NW-W     0.105719
W-W      0.088176
Name: overlap, dtype: float64

In [5]:
def jaccardIndex(list1,list2):
    set1=set(list1)
    set2=set(list2)
    intersection = set1.intersection(set2)
    tamIntersection=len(list(intersection))
    union = set1.union(set2)
    tamUnion=len(list(union))
    similitud=float(tamIntersection)/tamUnion
    return similitud

In [18]:
def comparePairsDF(df,func,qy):
    
    parAnalysis=[]
    categories=[]
    index=[]
    questionYear=[]
    
    idsAnalisis=list(set(df["repYear"]))
    cat1=""
    cat2=""
    
    for i in range(len(idsAnalisis)-1):
        print(idsAnalisis)
        print("----------------------")
        ident=idsAnalisis[i]
        operacionesIdent1=df[df["repYear"]==ident]['concept:name']
        #idsAnalisis.remove(ident)
        print(ident)
        print("----------------------")
        print("segunda lista")
        print(idsAnalisis)
        if ident in winners:
            cat1="W"
        else:
            cat1="NW"

        for j in range(i+1,len(idsAnalisis)-1):
            ident2=idsAnalisis[j]
            print("identificador dos")
            print(ident2)

            operacionesIdent2=df[df["repYear"]==ident2]['concept:name']

            if ident2 in winners:
                cat2="W"
            else:
                cat2="NW"

            cats=cat1+"-"+cat2

            categories.append(cats)
            index.append(func(operacionesIdent1,operacionesIdent2))
            parAnalysis.append(ident+"-"+ident2)
            questionYear.append(qy)
            
    dict_df={"parAnalisis":parAnalysis,"Categories":categories,"Comparison index":index,"questionYear":questionYear}
    df_analisis=pd.DataFrame.from_dict(dict_df)
    #print(df_analisis)
    return df_analisis

In [19]:
agrupaciones_años_preguntas=df_categorias_autores.groupby(["Question","Year"])
dict_df_analysis={"parAnalisis":[],"Categories":[],"Comparison index":[],"questionYear":[]}
df_analisis_similarity=pd.DataFrame.from_dict(dict_df_analysis)

for name, df_group in agrupaciones_años_preguntas:
    #print(df_group)
    df_año_pregunta=comparePairsDF(df_group,overlapIndex,name)
    df_analisis_similarity=pd.concat([df_analisis_similarity,df_año_pregunta])

['A17 2017', 'A11 2017', 'A18 2017', 'A2 2017', 'A10 2017', 'A8 2017', 'A6 2017', 'A5 2017', 'A14 2017', 'A12 2017', 'A1 2017', 'A22 2017', 'A15 2017', 'A16 2017', 'A21 2017', 'A7 2017', 'A23 2017', 'A9 2017', 'A13 2017', 'A3 2017', 'A19 2017']
----------------------
A17 2017
----------------------
segunda lista
['A17 2017', 'A11 2017', 'A18 2017', 'A2 2017', 'A10 2017', 'A8 2017', 'A6 2017', 'A5 2017', 'A14 2017', 'A12 2017', 'A1 2017', 'A22 2017', 'A15 2017', 'A16 2017', 'A21 2017', 'A7 2017', 'A23 2017', 'A9 2017', 'A13 2017', 'A3 2017', 'A19 2017']
identificador dos
A11 2017
identificador dos
A18 2017
identificador dos
A2 2017
identificador dos
A10 2017
identificador dos
A8 2017
identificador dos
A6 2017
identificador dos
A5 2017
identificador dos
A14 2017
identificador dos
A12 2017
identificador dos
A1 2017
identificador dos
A22 2017
identificador dos
A15 2017
identificador dos
A16 2017
identificador dos
A21 2017
identificador dos
A7 2017
identificador dos
A23 2017
identificador d

identificador dos
A26 2020
identificador dos
A37 2020
identificador dos
A16 2020
identificador dos
A18 2020
identificador dos
A12 2020
identificador dos
A13 2020
['A7 2020', 'A34 2020', 'A11 2020', 'A19 2020', 'A2 2020', 'A24 2020', 'A31 2020', 'A10 2020', 'A25 2020', 'A35 2020', 'A28 2020', 'A30 2020', 'A26 2020', 'A37 2020', 'A16 2020', 'A18 2020', 'A12 2020', 'A13 2020', 'A1 2020']
----------------------
A31 2020
----------------------
segunda lista
['A7 2020', 'A34 2020', 'A11 2020', 'A19 2020', 'A2 2020', 'A24 2020', 'A31 2020', 'A10 2020', 'A25 2020', 'A35 2020', 'A28 2020', 'A30 2020', 'A26 2020', 'A37 2020', 'A16 2020', 'A18 2020', 'A12 2020', 'A13 2020', 'A1 2020']
identificador dos
A10 2020
identificador dos
A25 2020
identificador dos
A35 2020
identificador dos
A28 2020
identificador dos
A30 2020
identificador dos
A26 2020
identificador dos
A37 2020
identificador dos
A16 2020
identificador dos
A18 2020
identificador dos
A12 2020
identificador dos
A13 2020
['A7 2020', 'A34 202

identificador dos
A26 2020
identificador dos
A31 2020
identificador dos
A28 2020
identificador dos
A37 2020
identificador dos
A11 2020
identificador dos
A16 2020
identificador dos
A18 2020
identificador dos
A19 2020
identificador dos
A12 2020
identificador dos
A13 2020
['A10 2020', 'A25 2020', 'A27 2020', 'A7 2020', 'A35 2020', 'A24 2020', 'A26 2020', 'A31 2020', 'A28 2020', 'A37 2020', 'A11 2020', 'A16 2020', 'A18 2020', 'A19 2020', 'A12 2020', 'A13 2020', 'A2 2020']
----------------------
A25 2020
----------------------
segunda lista
['A10 2020', 'A25 2020', 'A27 2020', 'A7 2020', 'A35 2020', 'A24 2020', 'A26 2020', 'A31 2020', 'A28 2020', 'A37 2020', 'A11 2020', 'A16 2020', 'A18 2020', 'A19 2020', 'A12 2020', 'A13 2020', 'A2 2020']
identificador dos
A27 2020
identificador dos
A7 2020
identificador dos
A35 2020
identificador dos
A24 2020
identificador dos
A26 2020
identificador dos
A31 2020
identificador dos
A28 2020
identificador dos
A37 2020
identificador dos
A11 2020
identificador

identificador dos
A16 2020
identificador dos
A12 2020
identificador dos
A13 2020
identificador dos
A1 2020
['A10 2020', 'A25 2020', 'A7 2020', 'A35 2020', 'A30 2020', 'A26 2020', 'A28 2020', 'A37 2020', 'A16 2020', 'A12 2020', 'A13 2020', 'A1 2020', 'A2 2020']
----------------------
A25 2020
----------------------
segunda lista
['A10 2020', 'A25 2020', 'A7 2020', 'A35 2020', 'A30 2020', 'A26 2020', 'A28 2020', 'A37 2020', 'A16 2020', 'A12 2020', 'A13 2020', 'A1 2020', 'A2 2020']
identificador dos
A7 2020
identificador dos
A35 2020
identificador dos
A30 2020
identificador dos
A26 2020
identificador dos
A28 2020
identificador dos
A37 2020
identificador dos
A16 2020
identificador dos
A12 2020
identificador dos
A13 2020
identificador dos
A1 2020
['A10 2020', 'A25 2020', 'A7 2020', 'A35 2020', 'A30 2020', 'A26 2020', 'A28 2020', 'A37 2020', 'A16 2020', 'A12 2020', 'A13 2020', 'A1 2020', 'A2 2020']
----------------------
A7 2020
----------------------
segunda lista
['A10 2020', 'A25 2020', '

In [20]:
df_analisis_similarity=df_analisis_similarity.reset_index()
df_analisis_similarity['Categories']=df_analisis_similarity['Categories'].replace("NW-W", "W-NW")

In [21]:
questionYearCategories=df_analisis_similarity.groupby("Categories").apply(lambda x: np.mean(x["Comparison index"]))
print(questionYearCategories)
#for key, item in questionYearCategories:
    #print(questionYearCategories.get_group(key),"\n\n")

Categories
NW-NW    0.477784
W-NW     0.599371
W-W      0.443333
dtype: float64


In [112]:
df_categorias_autores[df_categorias_autores["repYear"]=="A13 2017"]

,case:concept:name,concept:name,Specific operation,time:timestamp,Report,Question,Year,Category,Type,repYear
339,A13 P1 BPI 2017,Represent process map,Represent process map with frequency,2021-10-19 12:47:16.319135,A13,P1,2017,Student,Representation,A13 2017
340,A13 P1 BPI 2017,Represent histograms,Represent histograms of waiting time,2021-10-19 12:47:16.325118,A13,P1,2017,Student,Representation,A13 2017
341,A13 P1 BPI 2017,Calculate waiting time,,2021-10-19 12:47:16.330071,A13,P1,2017,Student,Analyze Cycle Time,A13 2017
342,A13 P1 BPI 2017,Identify bottlenecks,Identify sub-processes as bottlenecks applying...,2021-10-19 12:47:16.330494,A13,P1,2017,Student,Analyze Cycle Time,A13 2017
343,A13 P1 BPI 2017,Filter traces,Filter traces by activities,2021-10-19 12:47:16.343594,A13,P1,2017,Student,Manipulation,A13 2017
344,A13 P1 BPI 2017,Calculate number of elements,Calculate number of traces,2021-10-19 12:47:16.350147,A13,P1,2017,Student,Calculate statistics,A13 2017
345,A13 P1 BPI 2017,Represent process map,Represent process map with frequency,2021-10-19 12:47:16.360113,A13,P1,2017,Student,Representation,A13 2017


In [114]:
df_categorias_autores[df_categorias_autores["repYear"]=="A22 2017"]

,case:concept:name,concept:name,Specific operation,time:timestamp,Report,Question,Year,Category,Type,repYear
382,A22 P1 BPI 2017,Calculate waiting time,,2021-10-19 12:47:16.664286,A22,P1,2017,Academic,Analyze Cycle Time,A22 2017
383,A22 P1 BPI 2017,Assign resource to each activity,,2021-10-19 12:47:16.673255,A22,P1,2017,Academic,Others,A22 2017
384,A22 P1 BPI 2017,Calculate frequency,Calculate frequency of activities,2021-10-19 12:47:16.679239,A22,P1,2017,Academic,Calculate statistics,A22 2017
385,A22 P1 BPI 2017,Calculate frequency,Calculate frequency of activities,2021-10-19 12:47:16.686220,A22,P1,2017,Academic,Calculate statistics,A22 2017
386,A22 P1 BPI 2017,Identify bottlenecks,Identify activities as bottlenecks applying te...,2021-10-19 12:47:16.693239,A22,P1,2017,Academic,Analyze Cycle Time,A22 2017
